# Выбор модели для последующего трансферного обучения в PyThorch

## Задача

Выбрать оптимальную модель, предварительно обученную на наборе данных *ImageNet*: *VGG16*, *VGG19*, *ResNet50V1*, *ResNet50V2*, *ResNet101V1*, *ResNet101V2*, *ResNet152V1*, *ResNet152V2*.

## Данные

40 000 цветных картинок бетона размером 227x227 пикселей, 20 000 из которых с стрещиной, другие 20 000 целый  
Данные взяты из курса [AI Capstone Project with Deep Learning](https://www.coursera.org/learn/ai-deep-learning-capstone?specialization=ai-engineer), явлюющийся заключительным курсом [IBM AI Engineering Professional Certificate](https://www.coursera.org/professional-certificates/ai-engineer) на сайте [coursera.org](https://www.coursera.org/)  
[Данные](https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip)  

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip

--2023-04-28 12:12:00--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245259777 (234M) [application/zip]
Saving to: ‘concrete_crack_images_for_classification.zip’

concrete_crack_imag 100%[===================>] 233.90M  28.9MB/s    in 7.7s    

2023-04-28 12:12:09 (30.5 MB/s) - ‘concrete_crack_images_for_classification.zip’ saved [245259777/245259777]



In [2]:
!mkdir ./data
!mkdir ./data/concrete
!mkdir ./models

In [ ]:
!unzip concrete_crack_images_for_classification.zip -d ./data/concrete

## Расчёты

In [4]:
import os

from PIL import Image

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models

from tqdm import tqdm

In [5]:
class Concrete(Dataset):
    def __init__(self, train=True, train_size=0.75, transform=None):

        self.transform = transform

        # директории классов
        directory = "./data/concrete/"
        positive = "Positive"
        negative = "Negative"

        # путь к классам
        positive_path = os.path.join(directory, positive)
        negative_path = os.path.join(directory, negative)

        # путь к каждому положительному объекту
        positive_files = [os.path.join(positive_path, file) for file in os.listdir(positive_path) if file.endswith(".jpg")]
        positive_files.sort()

        # путь к каждому негативному объекту
        negative_files = [os.path.join(negative_path, file) for file in os.listdir(negative_path) if file.endswith(".jpg")]
        negative_files.sort()

        # пути к каждому объекту (чётные индексы - положительные классы, нечётные индексы - отрицательные классы)
        self.all_files = []
        for pair in zip(positive_files, negative_files):
            self.all_files.extend(pair)
        
        # всего объектов в данных
        length = len(self.all_files)

        # целевое обозначение каждого объекта 
        self.all_targets = torch.zeros(length, dtype=torch.long)
        self.all_targets[::2] = 1

        # индекс границы между данными обечения и данными валидации
        border = int(length * train_size)

        # разделение данных на тестовую валидационную выборки
        if train:
            self.all_files = self.all_files[:border]
            self.all_targets = self.all_targets[:border]
        else:
            self.all_files = self.all_files[border:]
            self.all_targets = self.all_targets[border:]
        
        self.len = len(self.all_targets)

    def __len__(self):
        return self.len

    def __getitem__(self, ind):
        
        # загрузка оъекта и его обозначения
        image = Image.open(self.all_files[ind])
        y = self.all_targets[ind]

        # трансформация объекта, если это необходимо
        if self.transform:
            image = self.transform(image)

        return image, y

In [6]:
def get_transform_model(name_model="VGG16"):

    # инициализация модели и её весов
    if name_model == "VGG16":
        
        weights = models.VGG16_Weights.IMAGENET1K_V1
        model = models.vgg16(weights=weights)
    
    elif name_model == "VGG19":

        weights = models.VGG19_Weights.IMAGENET1K_V1
        model = models.vgg19(weights=weights)
    
    elif name_model == "ResNet50V1":

        weights = models.ResNet50_Weights.IMAGENET1K_V1
        model = models.resnet50(weights=weights)

    elif name_model == "ResNet50V2":

        weights = models.ResNet50_Weights.IMAGENET1K_V2
        model = models.resnet50(weights=weights)

    elif name_model == "ResNet101V1":
        
        weights = models.ResNet101_Weights.IMAGENET1K_V1
        model = models.resnet101(weights=weights)

    elif name_model == "ResNet101V2":
        
        weights = models.ResNet101_Weights.IMAGENET1K_V2
        model = models.resnet101(weights=weights)

    elif name_model == "ResNet152V1":
        
        weights = models.ResNet152_Weights.IMAGENET1K_V1
        model = models.resnet152(weights=weights)

    elif name_model == "ResNet152V2":
        
        weights = models.ResNet152_Weights.IMAGENET1K_V2
        model = models.resnet152(weights=weights)


    # заморозка весов свёрточной модели
    for param in model.parameters():
        param.requires_grad = False


    # замена полносвязных слоёв в конце
    if name_model.startswith("VGG"):
        
        model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        model.classifier = nn.Linear(512, 2)

    elif name_model.startswith("ResNet"):
        
        model.fc = nn.Linear(2048, 2)

    # инициализация преобразователя данных для модели
    transform = weights.transforms()

    return transform, model

In [7]:
def get_batched_data(transform, train_size=0.75, batch_size=100, shuffle=True):
    
    # инициализация обучающих данных
    train = Concrete(train=True, train_size=train_size, transform=transform)
    # инициализация валидационных данных
    val = Concrete(train=False, train_size=train_size, transform=transform)
    
    # размер валидационной выборки
    val_len = len(val)

    # пакетированные данные для тренировки
    train = DataLoader(dataset=train, batch_size=batch_size, shuffle=shuffle)
    # пакетированные данные для валидации
    val = DataLoader(dataset=val, batch_size=batch_size, shuffle=shuffle)

    return train, val, val_len

In [8]:
def train(model, data_train, data_val, val_len, device='cpu', epochs=3, lr=0.001):
    
    # инициализация оптимизатора и функции ошибки
    optimizer = optim.Adam(params=[parameter for parameter in model.parameters() if parameter.requires_grad], lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # перевод модель на устройство для расчётов
    model.to(device)

    for epoch in range(epochs):
        
        # перевод модели в режим обучения
        model.train()

        # цикл обучения
        for x, y in tqdm(data_train, desc=f"Train epoch {epoch + 1: >3}"):

            # перевод данных на устройсто для расчётов         
            x, y = x.to(device), y.to(device)
            
            # обнуление градиента
            optimizer.zero_grad()
            
            # расчёт ошибки
            y_hat = model(x)
            loss = criterion(y_hat, y)

            # шаг обучения
            loss.backward()
            optimizer.step()

        # перевод модели в режим оценки
        model.eval()

        # число корректных ответов
        correct = 0

        # цикл оценки
        for x, y in tqdm(data_val, desc=f"Val epoch {epoch + 1: >5}"):

            # перевод данных на устройсто для расчётов 
            x, y = x.to(device), y.to(device)

            # сбор корректных ответов в батче
            y_hat = model(x)
            correct += (y_hat.argmax(dim=1) == y).sum().item()

        # расчёт точности
        accuracy = correct / val_len
        
        print(f"\nAccuracy: {accuracy * 100:.2f}%\n")

    # перевод модели на процессор
    model.to("cpu")
    print("="*40)


In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
all_names = ["VGG16", "VGG19", "ResNet50V1", "ResNet50V2", "ResNet101V1", "ResNet101V2", "ResNet152V1", "ResNet152V2"]


for name_model in all_names:
    
    print(f"\nModel: {name_model}\n")

    transform, model = get_transform_model(name_model=name_model)

    # вычисление числа параметров модели
    number_of_param = 0
    for param in model.parameters():
        number_of_param += param.numel()
    
    print(f"\nЧисло параметров сети: {number_of_param / 1e6:.3f} * 10^6")

    # параметры в формате float32, т.е. 1 параметр занимает в памяти 32 бита / 4 байта
    print(f"Занимают в памяти: {number_of_param * 4 / (1024 ** 2):.3f} Мб\n")

    data_train, data_val, val_len = get_batched_data(transform=transform,
                                                     train_size=0.75,
                                                     batch_size=100,
                                                     shuffle=True)

    train(model,
          data_train=data_train,
          data_val=data_val,
          val_len=val_len,
          device=device,
          epochs=3,
          lr=0.001)


Model: VGG16



Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 238MB/s]



Число параметров сети: 14.716 * 10^6
Занимают в памяти: 56.136 Мб



Val epoch     1: 100%|██████████| 100/100 [01:32<00:00,  1.08it/s]



Accuracy: 99.09%



...


Accuracy: 98.67%



Val epoch     2: 100%|██████████| 100/100 [01:46<00:00,  1.06s/it]



Accuracy: 99.14%



Val epoch     3: 100%|██████████| 100/100 [01:46<00:00,  1.07s/it]


Accuracy: 99.23%



## Результаты

Как видно, все модели, по истечению трёх эпох обучения, достигли точности свыше 99%. (Именно для этих данных)  
Так как помимо полносвязного слоя в конце, можно ещё слегка поднастроить веса свёрточных слоёв, то обращать внимание на сотые доли процента не имеет значения.  
Время обучения так же не является наиболее важным критерием для моделей, так как модель обучается единожды и потом это не требуется, к тому же все модели обучаются примерно за одно и тоже время.  
А вот количество параметров, а следовательно занимаемый объём памяти моделью, на мой взгляд более важный аспект.  
Из-за всего вышеперечисленного будем использовать самую легковесную модель - _**VGG16**_ с объёмом в ~ 56 Мб